In [1]:
import numpy as np

from dataset_preprocessing import create_dataset, extract_patches, compute_input_features, compute_output_features
from dataset_postprocessing import unpatch
from feature_extractors import extract_features, extract_features_2d
from visualization_helpers import *

%matplotlib inline

In [2]:
# GLOBAL VARIABLES
ROOT_DIR = "training/"
TOTAL_IMAGES = 10 # Number of images to load
TRAIN_FRACTION = 0.9 # Percentage of images used for training
ANGLE_STEP = 90 # Gotta be 360/ANGLE_STEP needs to be an integer
FLIP = True # Flag to signal if flipped  versions of rotated images should also be created
PATCH_SIZE = 200
PATCH_TRANSLATION = 100 # WARNING: this quickly explodes to enormous amount of data if small patch_translation is selected.
FOREGROUND_THRESHOLD = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
ORIGINAL_IMAGE_WIDTH = 400
ORIGINAL_IMAGE_HEIGHT = 400

In [3]:
dataset = create_dataset(ROOT_DIR, TOTAL_IMAGES, TRAIN_FRACTION, rotation_angle=ANGLE_STEP, flip=FLIP)

Original loaded dataset size: 10
Creating train dataset...
Number of rotations: 4
Flipping: True
Creating test dataset...
Number of rotations: 4
Flipping: True
Created train dataset size: 63
Created test dataset size: 1


In [4]:
patches = extract_patches(dataset, PATCH_SIZE, PATCH_TRANSLATION)

Train patches: 2268
Test patches: 36
Train GT patches: 2268
Test GT patches: 36


In [5]:
input_features = compute_input_features(patches[0:2], extract_features) # train_img_patches and test_img_patches
output_features = compute_output_features(patches[2:4], FOREGROUND_THRESHOLD) # train_gt_img_patches and test_gt_img_patches
                 
X = input_features[0]
Y = output_features[0]
Xi = input_features[1]
Yi = output_features[1]

Train features: 2268
Test features: 36
Train GT features: 2268
Test GT features: 36


In [6]:
# Print feature statistics
display_feature_statistics(X, Y)

Computed 2268 features
Feature dimension = 6
Number of classes = 1
Class 0: 14 samples
Class 1: 2254 samples


In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)  
X_train = scaler.transform(X)
clf = MLPClassifier()# default options
clf.fit(X_train, Y)
clf.score(X_train,Y)
Xi = scaler.transform(Xi)  
Zi = clf.predict(Xi)

In [8]:
unpached_Zi = unpatch(Zi, PATCH_SIZE, PATCH_TRANSLATION, ORIGINAL_IMAGE_WIDTH, ORIGINAL_IMAGE_HEIGHT)

/Users/mateusz/Dropbox/EPFL/Semester I/Machnie Learning CS-433/My Answers/MachineLearning/Projects/project2/project_road_segmentation/dataset_postprocessing.py:49: RuntimeWarning: invalid value encountered in true_divide
  unpatched_img = np.around((patched_image/index_image))[h_from:h_to, w_from:w_to]


In [12]:
unpached_Yi = unpatch(Yi, PATCH_SIZE, PATCH_TRANSLATION, ORIGINAL_IMAGE_WIDTH, ORIGINAL_IMAGE_HEIGHT)

/Users/mateusz/Dropbox/EPFL/Semester I/Machnie Learning CS-433/My Answers/MachineLearning/Projects/project2/project_road_segmentation/dataset_postprocessing.py:49: RuntimeWarning: invalid value encountered in true_divide
  unpatched_img = np.around((patched_image/index_image))[h_from:h_to, w_from:w_to]


In [15]:
from sklearn.metrics import f1_score

f1_score(Yi, Zi, average='binary')

1.0